In [1]:
import os
import time
import random
import grid2op
import numpy as np
import pandas as pd


DATA_PATH = '../training_data_track1'  # for demo only, use your own dataset 即使用的环境
SCENARIO_PATH = '../training_data_track1/chronics'
SAVE_PATH = './'
LINES2ATTACK = [45, 56, 0, 9, 13, 14, 18, 23, 27, 39]
NUM_EPISODES = 1000  # each scenario runs 100 times for each attack (or to say, sample 100 points)

env = grid2op.make(dataset=DATA_PATH, chronics_path=SCENARIO_PATH)
env.chronics_handler.shuffle()


array(['C:\\SoftwareFilePath\\Python_code\\L2RPN_NIPS_2020_a_PPO_Solution\\training_data_track1\\chronics\\Scenario_august_dummly',
       'C:\\SoftwareFilePath\\Python_code\\L2RPN_NIPS_2020_a_PPO_Solution\\training_data_track1\\chronics\\Scenario_february_dummy'],
      dtype='<U116')

In [2]:
episode=0
line_to_disconnect=45
env.reset()
dst_step = episode * 72 + random.randint(0, 72)  # a random sampling every 6 hours
print('\n\n' + '*' * 50 + '\nScenario[%s]: at step[%d], disconnect line-%d(from bus-%d to bus-%d]' % (
                    env.chronics_handler.get_name(), dst_step, line_to_disconnect,
                    env.line_or_to_subid[line_to_disconnect], env.line_ex_to_subid[line_to_disconnect]))



**************************************************
Scenario[Scenario_february_dummy]: at step[2], disconnect line-45(from bus-30 to bus-31]


In [3]:
env.fast_forward_chronics(dst_step - 1)
obs, reward, done, _ = env.step(env.action_space({}))
print('observation-rho:{}'.format(obs.rho))

observation-rho:[0.32687443 0.5535465  0.15082124 0.392285   0.5880148  0.2055941
 0.22000377 0.28082854 0.25977316 0.28769705 0.3221564  0.87557036
 0.510371   0.84048975 0.36942652 0.440951   0.21283472 0.14496417
 0.5547818  0.5547818  0.57550055 0.38268113 0.9084918  0.90736955
 0.51753277 0.5090575  0.38881963 0.5170754  0.51783544 0.19138987
 0.06299757 0.4798957  0.885958   0.89250934 0.8634984  0.8859001
 0.42496544 0.42947268 0.4292409  0.81567276 0.4912671  0.86378396
 0.47642875 0.26007125 0.13412376 0.619121   0.5006424  0.62390196
 0.36599004 0.36599004 0.5196889  0.51130414 0.5216738  0.40902066
 0.38994798 0.64865685 0.619121   0.63047916 0.7029415 ]


In [4]:
new_line_status_array = np.zeros(obs.rho.shape)
new_line_status_array[line_to_disconnect] = -1
print("obesercation.rho.shape:{}".format(obs.rho.shape))
print("new_line_status_array:{}".format(new_line_status_array))



obesercation.rho.shape:(59,)
new_line_status_array:[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]


In [5]:
action = env.action_space({"set_line_status": new_line_status_array})
obs, reward, done, _ = env.step(action)
print("action:{}".format(action))
print("action_dict:{}".format(action.as_dict()))

action:This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - force disconnection of 1 powerlines ([45])
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - NOT force any particular bus configuration
action_dict:{'set_line_status': {'nb_connected': 0, 'nb_disconnected': 1, 'connected_id': array([], dtype=int64), 'disconnected_id': array([45], dtype=int64)}}


In [6]:
all_actions = env.action_space.get_all_unitary_topologies_change(env.action_space)

In [7]:
print("all_actions[0]:{}".format(all_actions[0]))
print("all_actions[0]_asdict:{}".format(all_actions[0].as_dict()))
action1=all_actions[0]
# action1.as_dict()['change_bus_vect'][action.as_dict()['change_bus_vect']['modif_subs_id'][0]].items()
print("\n\n")
print("action1.as_dict()['change_bus_vect']['modif_subs_id']:{}".format(action1.as_dict()['change_bus_vect']['modif_subs_id'][0]))
print("\n\n")
print(action1.as_dict()['change_bus_vect'][action1.as_dict()['change_bus_vect']['modif_subs_id'][0]].items())

all_actions[0]:This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT force any line status
	 - NOT switch any line status
	 - Change the bus of the following element:
	 	 - switch bus of load 1 [on substation 0]
	 - NOT force any particular bus configuration
all_actions[0]_asdict:{'change_bus_vect': {'nb_modif_objects': 1, '0': {'1': {'type': 'load'}}, 'nb_modif_subs': 1, 'modif_subs_id': ['0']}}



action1.as_dict()['change_bus_vect']['modif_subs_id']:0



dict_items([('1', {'type': 'load'})])


In [8]:
print(action1.as_dict()['change_bus_vect']['0'].items())

dict_items([('1', {'type': 'load'})])


In [13]:
print("action1.to_vect(): {}".format(action1.to_vect()))
print("action1: {}".format(action1))
print("action1: {}".format(action1.attr_list_vect))

action1.to_vect(): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [10]:
print("obs_line: {}".format(obs.line_status))
disconnected_lines = np.where(obs.line_status == True)
print("disconnected_lines: {}".format(disconnected_lines))

obs_line: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True]
disconnected_lines: (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58], dtype=int64),)


In [11]:
print("obs.time_before_cooldown_line: {}".format(obs.time_before_cooldown_line))
print("len(os.listdir(SCENARIO_PATH)): {}".format(len(os.listdir(SCENARIO_PATH))))

obs.time_before_cooldown_line: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
len(os.listdir(SCENARIO_PATH)): 2


In [12]:
print("obs.to_vect().shape: {}".format(obs.to_vect().shape))
print("action1.to_vect().shape: {}".format(action1.to_vect().shape))

obs.to_vect().shape: (1266,)
action1.to_vect().shape: (494,)
